In [9]:
# KNN구현
# BMI 예제(multinomial classification)를 대상으로 KNN의 결과와
# LogisticRegression 결과 비교

import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

# raw data
df = pd.read_csv('./data/bmi.csv',skiprows=3)
display(df.head())

# data split
train_x_data, test_x_data, train_t_data, test_t_data=\
train_test_split(df[['height','weight']],
                 df['label'],
                 test_size=0.3,
                 random_state=1,
                 stratify=df['label'])

# 결측치X, 이상치X
# 정규화
scaler = MinMaxScaler()
scaler.fit(train_x_data)
norm_train_x_data = scaler.transform(train_x_data)
norm_test_x_data = scaler.transform(test_x_data)

# LogisticRegression 구현
model = LogisticRegression()
model.fit(norm_train_x_data, train_t_data)
acc = model.score(norm_test_x_data, test_t_data)
print('LogisticRegression의 accuracy : {}'.format(acc))  # 0.9851666666666666

# KNN
knn_classifier = KNeighborsClassifier(n_neighbors=3)
knn_classifier.fit(norm_train_x_data, train_t_data)
acc = knn_classifier.score(norm_test_x_data, test_t_data)
print('KNN의 accuracy : {}'.format(acc))  # 0.9985

,label,height,weight
0,1,188,71
1,2,161,68
2,0,178,52
3,2,136,63
4,1,145,52


LogisticRegression의 accuracy : 0.9851666666666666
KNN의 accuracy : 0.9985


In [1]:
%reset
# ozone랑 예측 linear regression 구현 (tensorflow 2.x)
# 데티어 전처리

import pandas as pd
import numpy as np
from scipy import stats
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
import warnings
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings(action='ignore')

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
#raw data
df = pd.read_csv('./data/ozone.csv')
display(df.head())

x_data = df[['Solar.R','Wind','Temp']]  # dataframe 2차원
t_data = df['Ozone']  # 1차원


,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
4,NaN,NaN,14.3,56,5,5


In [3]:
# 1. 먼저 독립변수에 대한 Missing Value를 찾아서 median으로 imputation

for col in x_data.columns:
    col_median = np.nanmedian(x_data[col])
    x_data[col].loc[x_data[col].isnull()] = col_median
    
# 2. 독립변수에 대한 이상치를 검출한 후 이상치를 제외한 나머지값들의 mean으로
#    이상치를 대체
zscore_threshold = 2.0

for col in x_data.columns:
    outlier = x_data[col][np.abs(stats.zscore(x_data[col])) > zscore_threshold]
    col_mean = np.mean(x_data.loc[~x_data[col].isin(outlier),col])
    x_data.loc[x_data[col].isin(outlier), col] = col_mean
    
    
# 3. 종속변수에 대한 이상치를 검출한 후 이상치를 제외한 나머지값들의 mean으로
#   이상치를 대체
outlier = t_data[np.abs(stats.zscore(t_data)) > zscore_threshold]
col_mean = np.mean(~t_data.isin(outlier))
t_data[t_data.isin(outlier)] = col_mean    

# 4. 정규화 진행
scaler_x = MinMaxScaler()
scaler_t = MinMaxScaler()

scaler_x.fit(x_data.values)               # scaler는 2차원 ndarray로 사용해야 해요!
scaler_t.fit(t_data.values.reshape(-1,1))  

norm_x_data = scaler_x.transform(x_data.values)
norm_t_data = scaler_t.transform(t_data.values.reshape(-1,1)).ravel()

# 5. 종속변수의 Missing Value는 KNN을 이용해서 예측값을 사용합니다.

# 종속변수가 Missing Value가 아닌 
# 독립변수들과 종속변수들을 추출(KNN을 학습하기 위해)
norm_train_x_data = norm_x_data[~np.isnan(norm_t_data)]
norm_train_t_data = norm_t_data[~np.isnan(norm_t_data)]

# 모델 생성
knn_regressor = KNeighborsRegressor(n_neighbors=2)
knn_regressor.fit(norm_train_x_data, norm_train_t_data)

# 종속변수가 Missing Value인 독립변수들을 입력으로 넣어서 값을 예측
knn_predict = knn_regressor.predict(norm_x_data[np.isnan(norm_t_data)])
norm_t_data[np.isnan(norm_t_data)] = knn_predict

In [4]:
# 최종적인 우리 데이터는
# norm_x_data
# norm_t_data

# 이제 데이터가 준비되었으니.. skearn 구현과 tensorflow 2.x으로 구현할꺼예요!

from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import SGD

test_data = np.array([[330, 15, 80]])   # 태양광 330, 바람 15, 온도 80

# sklearn 구현
model = LinearRegression()
model.fit(norm_x_data, norm_t_data)
result = model.predict(scaler_x.transform(test_data))

print('sklearn 예측값 : {}'.format(scaler_t.inverse_transform(result.reshape(-1,1))))
# sklearn 예측값 : [[30.12298524]]

# Tensorflow 2.x 구현(Linear Regression)
keras_model = Sequential()

keras_model.add(Flatten(input_shape=(3,)))  # Input Layer
keras_model.add(Dense(units=1,
                      activation='linear'))  # output Layer

keras_model.compile(optimizer=SGD(learning_rate=1e-2),
                    loss='mse')

keras_model.fit(norm_x_data,
                norm_t_data,
                epochs=5000,
                verbose=0)

result = keras_model.predict(scaler_x.transform(test_data))
print('tensorflow 예측값 : {}'.format(scaler_t.inverse_transform(result.reshape(-1,1))))
# tensorflow 예측값 : [[30.074213]]

sklearn 예측값 : [[36.93077619]]
tensorflow 예측값 : [[36.899017]]


In [1]:
%reset

# logistic regression
# binary classification을 sklearn과 tensorflow 2.x로 구현해보자
# titanic 데이터 사용 (kaggle에서 다운로드

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 결측치 처리 해야함, 단, 실제 데이터이기 때문에 이상치가 계산으로 검출된다 하더라도
# 검출된다 하다라도 해당 값을 그냥 사용할 꺼예요!

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
# raw data
df = pd.read_csv('./data/titanic/train.csv')

# 데이터 전처리

# 필요없는 칼럼 삭제처리
df = df.drop(['PassengerId','Name','Ticket','Fare','Cabin'],
             axis=1,
             inplace=False)

# display(df.head())

# 컬럼을 보고 하나로 합칠 수 있는 칼럼은 하나로 합쳐줘요!
df['Family'] = df['SibSp'] + df['Parch']
df= df.drop(['SibSp','Parch'], axis=1,inplace=False)
# display(df.head())

# 결측치 처리부터
df.isnull().sum()

# 'Embarked' column은 결측치가 2개
# 최빈값을 이용해서 missing value를 채워주자
df['Embarked'] = df['Embarked'].fillna('Q')

# 'Age' column의 결측치는 평균값으로 대체
df['Age'] = df['Age'].fillna(df['Age'].mean())

df.isnull().sum()

# 문자로 되어 있는 값을 숫자로 변경
gender_srting={'male' : 0, 'female':1}
df['Sex'] = df['Sex'].map(gender_srting)

embarked_string = {'S':0, 'C':1, 'Q':2}
df['Embarked'] = df['Embarked'].map(embarked_string)

def age_category(age):
    if((age >= 0) & (age < 25)):
        return 0
    elif ((age >=25) & (age < 50)):
        return 1
    else:
        return 2

df['Age'] = df['Age'].map(age_category)

df.head()

,Survived,Pclass,Sex,Age,Embarked,Family
0,0,3,0,0,0,1
1,1,1,1,1,1,1
2,1,3,1,1,0,0
3,1,1,1,1,0,1
4,0,3,0,1,0,0


In [3]:
# data split
train_x_data, test_x_data, train_t_data, test_t_data=\
train_test_split(df.drop('Survived', axis=1, inplace=False),
                 df['Survived'],
                 test_size=0.3,
                 random_state=1,
                 stratify=df['Survived'])

# 정규화
scaler = MinMaxScaler()
scaler.fit(train_x_data)
norm_train_x_data = scaler.transform(train_x_data)
norm_test_x_data = scaler.transform(test_x_data)

train_t_data.shape

(623,)

In [59]:
# sklearn 구현
model = LogisticRegression()
model.fit(norm_train_x_data, train_t_data)
sklearn_result = model.score(norm_test_x_data, test_t_data)
print('sklearn의 정확도 : {}'.format(sklearn_result))

sklearn의 정확도 : 0.7873134328358209


In [61]:
# tensorflow 구현
keras_model = Sequential()
keras_model.add(Flatten(input_shape=(5,)))
keras_model.add(Dense(units=1,
                      activation='sigmoid'))
keras_model.compile(optimizer=SGD(learning_rate=1e-2),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])
keras_model.fit(norm_train_x_data,
                train_t_data,
                epochs=1000,
                verbose=0)  # verbos=1은 학습하는걸 출력해서 보겠다는것
                            # verbose=0은 출력하지 않음
keras_result = keras_model.evaluate(norm_test_x_data,test_t_data)
print('Tensorflow 2.x 정확도 : {}'.format(keras_result))
# Tensorflow 2.x 정확도 : [0.4651029706001282, 0.7910447716712952]

9/9 [==============================] - 0s 998us/step - loss: 0.4651 - accuracy: 0.7910
Tensorflow 2.x 정확도 : [0.4651029706001282, 0.7910447716712952]
